# ЭТАП 5. Сборка и запуск ранее обученной модели

Заключительный этап данного курсового проекта: тестирование полученной модели на тестовых данных

In [2]:
import pandas as pd, dill
import numpy as np

In [3]:
with open('model_parts.dat','rb') as f:
    model_parts = dill.load(f)

pipeline = model_parts[0]
top_10 = model_parts[1]
model = model_parts[2]
del model_parts

In [4]:
#загружаем файл data_test.csv:
data_test = pd.read_csv('data_test.csv')
data_test = data_test[data_test.columns[1:]] #убираем столбец с номером строки

Готовим итоговый фрейм данных для модели. Очень долгий шаг из-за большого размера файла features.csv: время обработки даже для 10 признаков составляет ок. 2.5 часов

In [5]:
portion_n = 1           #начальная порция
portion_end = 100       #конечная порция
portion_strs = 100000   #число строк в порции

for i in range(portion_n,portion_end):

    print('Portion {}... '.format(i),end='')
    
    #Теперь извлечем признаки пользователей (первые 100000 строк):
    if i==1:    
        features = pd.read_csv('features.csv',nrows=portion_strs,delimiter='\t')
        features_columns = list(features.columns) #названия столбцов таблицы из исходного файла
    else:
        features = pd.read_csv('features.csv', nrows=portion_strs, skiprows=(i-1)*portion_strs+1, delimiter='\t', header=0, names=features_columns)
    
    features = features[features.columns[1:]]
    
    '''
    #отдельно обработаем признак с номером 252: он бинарный, но иногда попадаются
    #выбросы. Из-за этого кол-во столбцов после pipeline получается не всегда одним и тем же. Удалим соответствующие строки
    ind_incor = features.loc[((features['252']!=0) & (features['252']!=1))].index
    features.drop(ind_incor,inplace=True)
    '''
    
    features_left = features[['id','buy_time']] #левая часть features
    try:
        features = pipeline.fit_transform(features)
    except:
        print('Данные закончились')
        break
    
    
    #преобразование sparse-матрицы в DataFrame:
    #features = features.todense()
    features = pd.DataFrame(features)
    features = pd.concat((features_left,features),axis=1,ignore_index=True)
    
    #делаем left join таблиц data_train и features2
    features.columns = ['id','buy_time'] + top_10[3:]
    data_test2 = data_test.merge(features, how='left', on=['id','buy_time'])
    
    #Ранее мы определили, что в data_train пропусков нет. Поэтому сейчас для большинства
    #строк нет признаков. Поэтому удалим все строки, которые содержат nan-значения:
    data_test2.dropna(inplace=True)
        
    if i==1:
        data_test2.to_csv('prom_data.csv',mode='w',header=True,index=False)
    else:
        data_test2.to_csv('prom_data.csv',mode='a',header=False,index=False)
    
    print('ended')

Portion 1... ended
Portion 2... ended
Portion 3... ended
Portion 4... ended
Portion 5... ended
Portion 6... ended
Portion 7... ended
Portion 8... ended
Portion 9... ended
Portion 10... ended
Portion 11... ended
Portion 12... ended
Portion 13... ended
Portion 14... ended
Portion 15... ended
Portion 16... ended
Portion 17... ended
Portion 18... ended
Portion 19... ended
Portion 20... ended
Portion 21... ended
Portion 22... ended
Portion 23... ended
Portion 24... ended
Portion 25... ended
Portion 26... ended
Portion 27... ended
Portion 28... ended
Portion 29... ended
Portion 30... ended
Portion 31... ended
Portion 32... ended
Portion 33... ended
Portion 34... ended
Portion 35... ended
Portion 36... ended
Portion 37... ended
Portion 38... ended
Portion 39... ended
Portion 40... ended
Portion 41... ended
Portion 42... ended
Portion 43... ended
Portion 44... ended
Portion 45... ended
Portion 46... ended
Portion 47... Данные закончились


In [7]:
#убираем лишние переменные:
del data_test, data_test2, features, features_columns, features_left
del portion_end, portion_n, portion_strs

NameError: name 'data_test' is not defined

In [13]:
#загрузим и обработаем полученные данные:
data_test = pd.read_csv('prom_data.csv')
data_test_left = data_test[list(data_test.columns)[:3]] #информация о времени и услугах

data_test.head()

,id,vas_id,buy_time,59,221,123,37,26,194,80
0,2766218,2.0,1547413200,0.197067,-0.006631,-0.096139,-0.143489,0.503700,0.774382,-0.020189
1,3456548,5.0,1546808400,0.706854,-0.006631,-0.096139,0.095130,0.503700,0.774382,-0.020189
2,3456715,2.0,1547413200,0.536925,-0.006631,-0.096139,-0.130066,0.503700,-1.291353,-0.020189
3,1786063,6.0,1548018000,-0.652578,-0.006631,-0.096139,-0.143489,-1.985307,0.774382,-0.020189
4,3987315,5.0,1547413200,-0.482649,-0.006631,-0.096139,0.080216,0.503700,0.774382,-0.020189


In [14]:
#приведем тип услуги к dummy-виду и сконкатенируем датафреймы:
vas_id_test = pd.get_dummies(data_test.vas_id)
data_test = data_test[list(data_test.columns)[3:]]
data_test = pd.concat((vas_id_test[[6.0,4.0,9.0]],data_test),axis=1)

data_test.shape

(2048, 10)

In [17]:
#данные готовы, пропускаем через модель:
test_pred = model.predict_proba(data_test)
test_pred = np.array(test_pred[:,1] >= 0.5, dtype = np.uint8) #условно считаем порог равным 0.5
test_pred = pd.Series(test_pred, name = 'target')
print('Положительных ответов: {}'.format(test_pred.sum()))

#готовим итоговый датафрейм и сохраняем его в answers_test.csv
data_test_left = pd.concat((data_test_left,test_pred),axis=1)
data_test_left.to_csv('answers_test.csv')

Положительных ответов: 195


Из 2048 пользователей модель предсказала подключение услуг только для 195 из них (9,52%)